# Data

Import libraries:

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import catboost as cb
import gc

Import data & combine transaction and idendity columns:

In [2]:
train_transaction = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/train_transaction.csv')
train_identity = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/train_identity.csv')

train = pd.merge(train_transaction, train_identity, on="TransactionID", how="left")
train = train.set_index("TransactionID", drop="True")
del train_transaction, train_identity
gc.collect()

test_transaction = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/test_transaction.csv')
test_identity = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/test_identity.csv')

test = pd.merge(test_transaction, test_identity, on="TransactionID", how="left")
test = test.set_index("TransactionID", drop="True")
del test_transaction, test_identity
gc.collect()

0

In [3]:
train.shape

(590540, 433)

In [4]:
test.shape

(506691, 432)

In [5]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


Rename test data columns:

In [3]:
mapping = {}
for column_name in test.columns:
    mapping[column_name] = column_name.replace("-", "_")
test.rename(columns=mapping, inplace=True)

In [7]:
test.columns

Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=432)

Reduce memory usage:

In [4]:
import numpy as np


def reduce_mem_usage(df):
    """
    From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

    Iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            pass
            # df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 1955.37 MB
Memory usage after optimization is: 648.22 MB
Decreased by 66.8%
Memory usage of dataframe is 1673.87 MB
Memory usage after optimization is: 563.43 MB
Decreased by 66.3%


Split train into train/val:

In [5]:
VAL_SPLIT = 0.2
# Split train into train/val
y = train["isFraud"].copy()
X = train.drop("isFraud", axis=1)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=VAL_SPLIT, random_state=13
)

X_test = test.copy()
del train, test, X, y

X_train = X_train.fillna(-999)
X_val = X_val.fillna(-999)
X_test = X_test.fillna(-999)

Categorical features:

In [6]:
categorical_features = []

for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        categorical_features.append(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values) + list(X_val[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_val[f] = lbl.transform(list(X_val[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values)) 

# CatBoost

In [8]:
clf = cb.CatBoostClassifier(n_estimators=200,
                        learning_rate=0.05,
                        metric_period=500,
                        od_wait=500,
                        task_type='CPU',
                        depth=8)

In [10]:
clf.fit(X_train, y_train, cat_features=categorical_features)

0:	learn: 0.6113626	total: 145ms	remaining: 28.8s
199:	learn: 0.0947855	total: 26.2s	remaining: 0us


Results:

In [9]:
def calculate_scores(estimator, X_val, y_val):
    y_val_prediction = estimator.predict(X_val)
    y_val_proba = estimator.predict_proba(X_val)[:, 1]

    conf_matrix = metrics.confusion_matrix(y_val, y_val_prediction)
    accuracy_score = metrics.accuracy_score(y_val, y_val_prediction)
    roc_auc_score = metrics.roc_auc_score(y_val, y_val_proba)
    f1_score = metrics.f1_score(y_val, y_val_prediction)
    classification_report = metrics.classification_report(y_val, y_val_prediction)

    print("Confusion Matrix: \n%s" % str(conf_matrix))
    print("\nAccuracy: %.4f" % accuracy_score)
    print("\nAUC: %.4f" % roc_auc_score)
    print("\nF1 Score: %.4f" % f1_score)
    print("\nClassification Report: \n",classification_report)

    return {
        "conf_matrix": conf_matrix,
        "accuracy_score": accuracy_score,
        "roc_auc_score": roc_auc_score,
        "f1_score": f1_score,
        "classification_report": classification_report
    }

In [16]:
_ = calculate_scores(clf, X_val, y_val)

Confusion Matrix: 
[[113854    118]
 [  2787   1349]]

Accuracy: 0.9754

AUC: 0.8755

F1 Score: 0.4815

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    113972
           1       0.92      0.33      0.48      4136

    accuracy                           0.98    118108
   macro avg       0.95      0.66      0.73    118108
weighted avg       0.97      0.98      0.97    118108



# Hyperarameter Optimization

Randomized Search:

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt  

param_grid = {
    'silent': [False],
    'learning_rate': sp_randFloat(0.01, 0.3),
    'n_estimators': [200],
    'depth': sp_randInt(6, 16),
    'l2_leaf_reg':[3,1,5,10,100],
    'loss_function': ['Logloss', 'CrossEntropy']
}

def perform_random_search(
    estimator, X_train, X_val, y_train, y_val, param_grid, scoring=None
):
    hyperparam_optimizer = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        scoring=scoring,
        cv=2,
        n_iter=20,
        n_jobs=1,
        refit=True,
        random_state=13,
    )
    hyperparam_optimizer.fit(X_train, y_train, eval_set=(X_val, y_val))

    return hyperparam_optimizer.best_estimator_

In [11]:
# accuracy score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='accuracy')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3729272	test: 0.3732352	best: 0.3732352 (0)	total: 134ms	remaining: 26.6s
199:	learn: 0.0609253	test: 0.0762992	best: 0.0762992 (199)	total: 16s	remaining: 0us

bestTest = 0.07629918512
bestIteration = 199



0:	learn: 0.3717054	test: 0.3720828	best: 0.3720828 (0)	total: 77.2ms	remaining: 15.4s
199:	learn: 0.0618371	test: 0.0758618	best: 0.0758618 (199)	total: 16s	remaining: 0us

bestTest = 0.07586180293
bestIteration = 199



0:	learn: 0.4354961	test: 0.4357128	best: 0.4357128 (0)	total: 74.7ms	remaining: 14.9s
199:	learn: 0.0797422	test: 0.0853030	best: 0.0853030 (199)	total: 15.9s	remaining: 0us

bestTest = 0.0853030195
bestIteration = 199



0:	learn: 0.4342225	test: 0.4344876	best: 0.4344876 (0)	total: 76.7ms	remaining: 15.3s
199:	learn: 0.0840276	test: 0.0885463	best: 0.0885463 (199)	total: 14.4s	remaining: 0us

bestTest = 0.08854631176
bestIteration = 199



0:	learn: 0.1663067	test: 0.1666409	best: 0.1666409 (0)	total: 119ms	remaining: 23.6s
199:	learn: 0.0397854	test: 0.0735952	best: 0.0735952 (199)	total: 23.6s	remaining: 0us

bestTest = 0.07359518541
bestIteration = 199



...

0:	learn: 0.1719746	test: 0.1726591	best: 0.1726591 (0)	total: 687ms	remaining: 2m 16s
199:	learn: 0.0096104	test: 0.0816150	best: 0.0770208 (91)	total: 2m 19s	remaining: 0us

bestTest = 0.07702082961
bestIteration = 91

Shrink model to first 92 iterations.


0:	learn: 0.1722226	test: 0.1734552	best: 0.1734552 (0)	total: 690ms	remaining: 2m 17s
199:	learn: 0.0095227	test: 0.0814562	best: 0.0768382 (78)	total: 2m 20s	remaining: 0us

bestTest = 0.07683818156
bestIteration = 78

Shrink model to first 79 iterations.


0:	learn: 0.1334633	test: 0.1338029	best: 0.1338029 (0)	total: 57.3ms	remaining: 11.4s
199:	learn: 0.0525424	test: 0.0768041	best: 0.0768041 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07680409854
bestIteration = 199



0:	learn: 0.1324851	test: 0.1334653	best: 0.1334653 (0)	total: 54.3ms	remaining: 10.8s
199:	learn: 0.0528575	test: 0.0781396	best: 0.0781357 (196)	total: 11.2s	remaining: 0us

bestTest = 0.07813565154
bestIteration = 196

Shrink model to first 197 iterations.


0:	learn: 0.1734551	test: 0.1746899	best: 0.1746899 (0)	total: 2.78s	remaining: 9m 13s
199:	learn: 0.0159368	test: 0.0550455	best: 0.0550294 (195)	total: 9m 15s	remaining: 0us

bestTest = 0.05502935718
bestIteration = 195

Shrink model to first 196 iterations.
Confusion Matrix: 
[[113824    148]
 [  1559   2577]]

Accuracy: 0.9855

AUC: 0.9570

F1 Score: 0.7512

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    113972
           1       0.95      0.62      0.75      4136

    accuracy                           0.99    118108
   macro avg       0.97      0.81      0.87    118108
weighted avg       0.99      0.99      0.98    118108



In [12]:
# roc auc score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='roc_auc')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3729272	test: 0.3732352	best: 0.3732352 (0)	total: 76.1ms	remaining: 15.1s
199:	learn: 0.0609253	test: 0.0762992	best: 0.0762992 (199)	total: 15.2s	remaining: 0us

bestTest = 0.07629918512
bestIteration = 199



0:	learn: 0.3717054	test: 0.3720828	best: 0.3720828 (0)	total: 72.6ms	remaining: 14.4s
199:	learn: 0.0618371	test: 0.0758618	best: 0.0758618 (199)	total: 15.1s	remaining: 0us

bestTest = 0.07586180293
bestIteration = 199



0:	learn: 0.4354961	test: 0.4357128	best: 0.4357128 (0)	total: 72.4ms	remaining: 14.4s
199:	learn: 0.0797422	test: 0.0853030	best: 0.0853030 (199)	total: 14.4s	remaining: 0us

bestTest = 0.0853030195
bestIteration = 199



0:	learn: 0.4342225	test: 0.4344876	best: 0.4344876 (0)	total: 72.3ms	remaining: 14.4s
199:	learn: 0.0840276	test: 0.0885463	best: 0.0885463 (199)	total: 13.6s	remaining: 0us

bestTest = 0.08854631176
bestIteration = 199



0:	learn: 0.1663067	test: 0.1666409	best: 0.1666409 (0)	total: 109ms	remaining: 21.7s
199:	learn: 0.0397854	test: 0.0735952	best: 0.0735952 (199)	total: 22.1s	remaining: 0us

bestTest = 0.07359518541
bestIteration = 199



...

0:	learn: 0.1719746	test: 0.1726591	best: 0.1726591 (0)	total: 698ms	remaining: 2m 18s
199:	learn: 0.0096104	test: 0.0816150	best: 0.0770208 (91)	total: 2m 20s	remaining: 0us

bestTest = 0.07702082961
bestIteration = 91

Shrink model to first 92 iterations.


0:	learn: 0.1722226	test: 0.1734552	best: 0.1734552 (0)	total: 696ms	remaining: 2m 18s
199:	learn: 0.0095227	test: 0.0814562	best: 0.0768382 (78)	total: 2m 21s	remaining: 0us

bestTest = 0.07683818156
bestIteration = 78

Shrink model to first 79 iterations.


0:	learn: 0.1334633	test: 0.1338029	best: 0.1338029 (0)	total: 55.1ms	remaining: 11s
199:	learn: 0.0525424	test: 0.0768041	best: 0.0768041 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07680409854
bestIteration = 199



0:	learn: 0.1324851	test: 0.1334653	best: 0.1334653 (0)	total: 56.8ms	remaining: 11.3s
199:	learn: 0.0528575	test: 0.0781396	best: 0.0781357 (196)	total: 11.2s	remaining: 0us

bestTest = 0.07813565154
bestIteration = 196

Shrink model to first 197 iterations.


0:	learn: 0.4035852	test: 0.4042107	best: 0.4042107 (0)	total: 1.54s	remaining: 5m 6s
199:	learn: 0.0351829	test: 0.0572389	best: 0.0572389 (199)	total: 5m 5s	remaining: 0us

bestTest = 0.05723890246
bestIteration = 199

Confusion Matrix: 
[[113879     93]
 [  1822   2314]]

Accuracy: 0.9838

AUC: 0.9563

F1 Score: 0.7073

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    113972
           1       0.96      0.56      0.71      4136

    accuracy                           0.98    118108
   macro avg       0.97      0.78      0.85    118108
weighted avg       0.98      0.98      0.98    118108



In [13]:
# f1 score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='f1')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.5623241	test: 0.5624611	best: 0.5624611 (0)	total: 138ms	remaining: 27.5s
199:	learn: 0.0765317	test: 0.0838029	best: 0.0838029 (199)	total: 15.8s	remaining: 0us

bestTest = 0.08380294246
bestIteration = 199



0:	learn: 0.5617113	test: 0.5618594	best: 0.5618594 (0)	total: 78.2ms	remaining: 15.6s
199:	learn: 0.0773101	test: 0.0843051	best: 0.0843051 (199)	total: 15.8s	remaining: 0us

bestTest = 0.08430514911
bestIteration = 199



0:	learn: 0.5889800	test: 0.5890728	best: 0.5890728 (0)	total: 73.7ms	remaining: 14.7s
199:	learn: 0.0880873	test: 0.0915331	best: 0.0915331 (199)	total: 15.5s	remaining: 0us

bestTest = 0.09153312308
bestIteration = 199



0:	learn: 0.5883829	test: 0.5884874	best: 0.5884874 (0)	total: 77.7ms	remaining: 15.5s
199:	learn: 0.0881391	test: 0.0915712	best: 0.0915712 (199)	total: 15.7s	remaining: 0us

bestTest = 0.09157118676
bestIteration = 199



0:	learn: 0.4188817	test: 0.4191061	best: 0.4191061 (0)	total: 117ms	remaining: 23.3s
199:	learn: 0.0603543	test: 0.0751775	best: 0.0751775 (199)	total: 23.2s	remaining: 0us

bestTest = 0.07517748952
bestIteration = 199



...

0:	learn: 0.4266684	test: 0.4270493	best: 0.4270493 (0)	total: 688ms	remaining: 2m 16s
199:	learn: 0.0322814	test: 0.0669069	best: 0.0669069 (199)	total: 2m 17s	remaining: 0us

bestTest = 0.06690693757
bestIteration = 199



0:	learn: 0.4272732	test: 0.4277570	best: 0.4277570 (0)	total: 687ms	remaining: 2m 16s
199:	learn: 0.0305460	test: 0.0674009	best: 0.0674009 (199)	total: 2m 19s	remaining: 0us

bestTest = 0.06740089778
bestIteration = 199



0:	learn: 0.3506483	test: 0.3509457	best: 0.3509457 (0)	total: 54.5ms	remaining: 10.8s
199:	learn: 0.0678429	test: 0.0785547	best: 0.0785547 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07855467557
bestIteration = 199



0:	learn: 0.3493491	test: 0.3497539	best: 0.3497539 (0)	total: 53.7ms	remaining: 10.7s
199:	learn: 0.0695557	test: 0.0795903	best: 0.0795903 (199)	total: 11.2s	remaining: 0us

bestTest = 0.07959034441
bestIteration = 199



0:	learn: 0.4205230	test: 0.4212135	best: 0.4212135 (0)	total: 912ms	remaining: 3m 1s
199:	learn: 0.0366199	test: 0.0582201	best: 0.0582201 (199)	total: 3m 7s	remaining: 0us

bestTest = 0.05822014778
bestIteration = 199

Confusion Matrix: 
[[113878     94]
 [  1822   2314]]

Accuracy: 0.9838

AUC: 0.9546

F1 Score: 0.7072

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    113972
           1       0.96      0.56      0.71      4136

    accuracy                           0.98    118108
   macro avg       0.97      0.78      0.85    118108
weighted avg       0.98      0.98      0.98    118108



Bayesian Search:

In [10]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

param_grid = {
    'silent': [False],
    'learning_rate': Real(0.01, 0.3),
    'n_estimators': [200],
    'depth': Integer(6, 16),
    'l2_leaf_reg':[3,1,5,10,100],
    'loss_function': ['Logloss', 'CrossEntropy']
}

def perform_bayes_search(
    estimator, X_train, X_val, y_train, y_val, param_grid, scoring=None
):
    hyperparam_optimizer = BayesSearchCV(
        estimator=estimator,
        search_spaces=param_grid,
        scoring=scoring,
        cv=2,
        n_iter=15,
        n_jobs=1,
        refit=True,
        return_train_score=False,
        optimizer_kwargs={"base_estimator": "GP"},
        random_state=13,
        fit_params={
                'eval_set': (X_val, y_val),
        }
    )
    hyperparam_optimizer.fit(X_train, y_train)

    return hyperparam_optimizer.best_estimator_

In [11]:
# accuracy score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='accuracy')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3632505	test: 0.3635394	best: 0.3635394 (0)	total: 109ms	remaining: 21.8s
199:	learn: 0.0690067	test: 0.0788914	best: 0.0788914 (199)	total: 12.1s	remaining: 0us

bestTest = 0.07889137956
bestIteration = 199



0:	learn: 0.3619733	test: 0.3623615	best: 0.3623615 (0)	total: 59.1ms	remaining: 11.8s
199:	learn: 0.0699066	test: 0.0796372	best: 0.0796372 (199)	total: 12.1s	remaining: 0us

bestTest = 0.07963721797
bestIteration = 199



0:	learn: 0.3589374	test: 0.3592340	best: 0.3592340 (0)	total: 468ms	remaining: 1m 33s
199:	learn: 0.0469093	test: 0.0700253	best: 0.0700253 (199)	total: 1m 32s	remaining: 0us

bestTest = 0.07002527255
bestIteration = 199



0:	learn: 0.3571715	test: 0.3575993	best: 0.3575993 (0)	total: 462ms	remaining: 1m 31s
199:	learn: 0.0497374	test: 0.0714763	best: 0.0714763 (199)	total: 1m 32s	remaining: 0us

bestTest = 0.07147628797
bestIteration = 199



0:	learn: 0.6546726	test: 0.6547063	best: 0.6547063 (0)	total: 426ms	remaining: 1m 24s
199:	learn: 0.0850456	test: 0.0907384	best: 0.0907384 (199)	total: 1m 28s	remaining: 0us

bestTest = 0.0907383532
bestIteration = 199



...

0:	learn: 0.3265272	test: 0.3267374	best: 0.3267374 (0)	total: 43.4ms	remaining: 8.63s
199:	learn: 0.0668413	test: 0.0786039	best: 0.0786039 (199)	total: 9.3s	remaining: 0us

bestTest = 0.07860390144
bestIteration = 199



0:	learn: 0.3253090	test: 0.3255909	best: 0.3255909 (0)	total: 42.5ms	remaining: 8.46s
199:	learn: 0.0675265	test: 0.0788202	best: 0.0788202 (199)	total: 9.27s	remaining: 0us

bestTest = 0.0788201767
bestIteration = 199



0:	learn: 0.3642152	test: 0.3647106	best: 0.3647106 (0)	total: 9.24s	remaining: 30m 39s
199:	learn: 0.0112147	test: 0.0650428	best: 0.0645113 (158)	total: 31m 23s	remaining: 0us

bestTest = 0.06451131912
bestIteration = 158

Shrink model to first 159 iterations.


0:	learn: 0.3671447	test: 0.3676809	best: 0.3676809 (0)	total: 9.37s	remaining: 31m 4s
199:	learn: 0.0112508	test: 0.0673803	best: 0.0672623 (186)	total: 31m 38s	remaining: 0us

bestTest = 0.06726233505
bestIteration = 186

Shrink model to first 187 iterations.


0:	learn: 0.3590043	test: 0.3597418	best: 0.3597418 (0)	total: 10.2s	remaining: 33m 57s
199:	learn: 0.0137726	test: 0.0510194	best: 0.0509922 (196)	total: 32m 58s	remaining: 0us

bestTest = 0.05099219388
bestIteration = 196

Shrink model to first 197 iterations.
Confusion Matrix: 
[[113872    100]
 [  1635   2501]]

Accuracy: 0.9853

AUC: 0.9656

F1 Score: 0.7425

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    113972
           1       0.96      0.60      0.74      4136

    accuracy                           0.99    118108
   macro avg       0.97      0.80      0.87    118108
weighted avg       0.98      0.99      0.98    118108



In [12]:
# roc auc score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='roc_auc')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3632505	test: 0.3635394	best: 0.3635394 (0)	total: 54.3ms	remaining: 10.8s
199:	learn: 0.0690067	test: 0.0788914	best: 0.0788914 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07889137956
bestIteration = 199



0:	learn: 0.3619733	test: 0.3623615	best: 0.3623615 (0)	total: 54.3ms	remaining: 10.8s
199:	learn: 0.0699066	test: 0.0796372	best: 0.0796372 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07963721797
bestIteration = 199



0:	learn: 0.3589374	test: 0.3592340	best: 0.3592340 (0)	total: 427ms	remaining: 1m 25s
199:	learn: 0.0469093	test: 0.0700253	best: 0.0700253 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.07002527255
bestIteration = 199



0:	learn: 0.3571715	test: 0.3575993	best: 0.3575993 (0)	total: 426ms	remaining: 1m 24s
199:	learn: 0.0497374	test: 0.0714763	best: 0.0714763 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.07147628797
bestIteration = 199



0:	learn: 0.6546726	test: 0.6547063	best: 0.6547063 (0)	total: 424ms	remaining: 1m 24s
199:	learn: 0.0850456	test: 0.0907384	best: 0.0907384 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.0907383532
bestIteration = 199



...

0:	learn: 0.4963730	test: 0.4966845	best: 0.4966845 (0)	total: 9.26s	remaining: 30m 42s
199:	learn: 0.0121035	test: 0.0664060	best: 0.0663089 (166)	total: 31m 30s	remaining: 0us

bestTest = 0.06630890944
bestIteration = 166

Shrink model to first 167 iterations.


0:	learn: 0.4970489	test: 0.4974700	best: 0.4974700 (0)	total: 9.3s	remaining: 30m 51s
199:	learn: 0.0120657	test: 0.0684628	best: 0.0682128 (147)	total: 31m 34s	remaining: 0us

bestTest = 0.06821282322
bestIteration = 147

Shrink model to first 148 iterations.


0:	learn: 0.3221547	test: 0.3223672	best: 0.3223672 (0)	total: 43.6ms	remaining: 8.69s
199:	learn: 0.0669088	test: 0.0787969	best: 0.0787969 (199)	total: 9.26s	remaining: 0us

bestTest = 0.07879689519
bestIteration = 199



0:	learn: 0.3209318	test: 0.3212195	best: 0.3212195 (0)	total: 44ms	remaining: 8.75s
199:	learn: 0.0669675	test: 0.0791311	best: 0.0791311 (199)	total: 9.32s	remaining: 0us

bestTest = 0.07913114938
bestIteration = 199



0:	learn: 0.4614521	test: 0.4619493	best: 0.4619493 (0)	total: 9.69s	remaining: 32m 8s
199:	learn: 0.0212907	test: 0.0516750	best: 0.0516750 (199)	total: 33m 2s	remaining: 0us

bestTest = 0.05167500448
bestIteration = 199

Confusion Matrix: 
[[113885     87]
 [  1730   2406]]

Accuracy: 0.9846

AUC: 0.9669

F1 Score: 0.7259

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    113972
           1       0.97      0.58      0.73      4136

    accuracy                           0.98    118108
   macro avg       0.98      0.79      0.86    118108
weighted avg       0.98      0.98      0.98    118108



In [ ]:
# f1 score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='f1')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3632505	test: 0.3635394	best: 0.3635394 (0)	total: 55.2ms	remaining: 11s
199:	learn: 0.0690067	test: 0.0788914	best: 0.0788914 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07889137956
bestIteration = 199



0:	learn: 0.3619733	test: 0.3623615	best: 0.3623615 (0)	total: 54.4ms	remaining: 10.8s
199:	learn: 0.0699066	test: 0.0796372	best: 0.0796372 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07963721797
bestIteration = 199



0:	learn: 0.3589374	test: 0.3592340	best: 0.3592340 (0)	total: 426ms	remaining: 1m 24s
199:	learn: 0.0469093	test: 0.0700253	best: 0.0700253 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.07002527255
bestIteration = 199



0:	learn: 0.3571715	test: 0.3575993	best: 0.3575993 (0)	total: 419ms	remaining: 1m 23s
199:	learn: 0.0497374	test: 0.0714763	best: 0.0714763 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.07147628797
bestIteration = 199



0:	learn: 0.6546726	test: 0.6547063	best: 0.6547063 (0)	total: 418ms	remaining: 1m 23s
199:	learn: 0.0850456	test: 0.0907384	best: 0.0907384 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.0907383532
bestIteration = 199



...

0:	learn: 0.3745195	test: 0.3749221	best: 0.3749221 (0)	total: 417ms	remaining: 1m 23s
199:	learn: 0.0514214	test: 0.0723493	best: 0.0723493 (199)	total: 1m 24s	remaining: 0us

bestTest = 0.07234927369
bestIteration = 199



0:	learn: 0.3364419	test: 0.3369166	best: 0.3369166 (0)	total: 9.25s	remaining: 30m 40s
199:	learn: 0.0144311	test: 0.0641426	best: 0.0641179 (194)	total: 31m 26s	remaining: 0us

bestTest = 0.06411791361
bestIteration = 194

Shrink model to first 195 iterations.


0:	learn: 0.3388416	test: 0.3394037	best: 0.3394037 (0)	total: 9.43s	remaining: 31m 16s
199:	learn: 0.0170804	test: 0.0654920	best: 0.0654537 (195)	total: 31m 32s	remaining: 0us

bestTest = 0.06545369112
bestIteration = 195

Shrink model to first 196 iterations.


0:	learn: 0.3086983	test: 0.3093487	best: 0.3093487 (0)	total: 9.32s	remaining: 30m 55s
199:	learn: 0.0025595	test: 0.0762207	best: 0.0698706 (78)	total: 31m 26s	remaining: 0us

bestTest = 0.06987060834
bestIteration = 78

Shrink model to first 79 iterations.


0:	learn: 0.3094932	test: 0.3104286	best: 0.3104286 (0)	total: 9.22s	remaining: 30m 35s
